In [200]:
!pip install selenium
!pip install pandas

You should consider upgrading via the 'c:\users\guilherme\outros_repositorios\botwhatsapp\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\guilherme\outros_repositorios\botwhatsapp\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [201]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import pandas

In [202]:
class Bot():
    
    def __init__(self):
        
        # Inicia o Browser indo diretamente para o site do Whatsapp, 
        # aguarda a resposta do QR Code por 60 segundos 
        # ou até encontrar a barra de pesquisa
        
        chrome_options = webdriver.ChromeOptions()
        self.wd = webdriver.Chrome(executable_path=r'./chromedriver.exe', 
                                   options=chrome_options)
        
        self.wd.get("https://web.whatsapp.com/")
        WebDriverWait(self.wd, 60).until(EC.element_to_be_clickable((By.ID, 
                                                                     'side')))
        
        try:
            
            # Inicia classe que busca um arquivo excel e com 
            # uma função que gera um dicionário com 
            # os dois parametros passados
            
            self.contatos = self.PData().ache_contatos("Nome", "Celular")
            print(self.contatos)
            
        except:
            print('Algo de errado com a planilha!')
    
    

    def texto_mensagem(self):
        
        # Função recebe um dicionário com nome e telefone 
        # que retorna a mensagem com o primeiro nome do contato
        # e o telefone de contato
        
        for index in self.contatos.values():
            yield 'Oi {}, isso é um teste automatizado'.format(index['Nome'].split(' ')[0]), index['Nome'], index['Celular']


    def nova_mensagem(self):
        
        # Função pesquisa o contato usando o numero na barra de pesquisa
        # e envia a mensagem
        
        for mensagem, nome, numero in self.texto_mensagem():
            
            BARRA_DE_PESQUISA = "//*[@id='side']/div[1]/div/label/div/div[2]"
            BOTÃO_ENVIAR = "//*[@id='main']/footer/div[1]/div[3]"
            DIGITE_SUA_MENSAGEM = "//*[@id='main']/footer/div[1]/div[2]/div/div[2]"
            BOTÃO_NOVA_MENSAGEM = "//*[@id='side']/header/div[2]/div/span/div[2]/div/span"
            CAIXA_NOVA_MENSAGEM = "//*[@id='app']/div/div/div[2]/div[1]/span/div/span/div/div[1]/div/label/div/div[2]"
            PRIMEIRO_CONTATO_PESQUISADO = "//*[@id='app']/div/div/div[2]/div[1]/span/div/span/div/div[2]/div[1]/div/div/div[2]/div/div/div[2]/div[1]/div/span/span"
            BOTÃO_NOVA_MENSAGEM_VOLTAR = "//*[@id='app']/div/div/div[2]/div[1]/span/div/span/div/header/div/div[1]/button/span"
            
            
            pesquisa = self.wd.find_element_by_xpath(BOTÃO_NOVA_MENSAGEM)
            pesquisa.click()
            
            time.sleep(2)
            
            caixa_nov_msg = self.wd.find_element_by_xpath(CAIXA_NOVA_MENSAGEM)
            caixa_nov_msg.send_keys(f"{numero}")
            
            time.sleep(2)
            
            try:
                WebDriverWait(self.wd, 2).until(EC.element_to_be_clickable((By.XPATH, PRIMEIRO_CONTATO_PESQUISADO)))
            except:
                voltar = self.wd.find_element_by_xpath(BOTÃO_NOVA_MENSAGEM_VOLTAR)
                voltar.click()
                continue
            
            time.sleep(2)
            
            contato = self.wd.find_element_by_xpath(PRIMEIRO_CONTATO_PESQUISADO)
            contato.click()
            
            digite = self.wd.find_element_by_xpath(DIGITE_SUA_MENSAGEM)
            digite.send_keys(f"Olá, esta é uma mensagem automática de um bot que está sendo alimentado por uma tabela com 'Nomes' e 'Numeros', o seu nome é: {nome} e o seu numero de telefone é: {numero}", Keys.RETURN)            

            
    class PData:

        def __init__(self):
            
            # Inicia a classe passando o diretório do Bot e 
            # passa uma lista de extensões de arquivos excel que 
            # serão filtrados e iterados
            
            self.pasta_excel = os.listdir(os.path.abspath(''))
            self.ext = ['.xlsx', 'xlsm', 'xltx', 'xltm']

        def filtrar_arquivo(self):
            
            # Filtra os arquivos para que seja aceito apenas arquivos excel
            
            for arquivo in self.pasta_excel:
                try:
                    if os.path.splitext(arquivo)[1] in self.ext:
                        return arquivo
                except :
                    print('Não foi encontrado arquivo excel nos formatos: {}'.format(self.ext))

                
        def ache_contatos(self, nom='', cel=''):
            
            # Dois parametros são utilizados para escolher os headers no excel que 
            # serão futuramente utilizados no dicionário para o Bot
            
            df = pandas.read_excel(self.filtrar_arquivo(), header=None, names=[nom, cel], skiprows=1, index_col=None)
            df.reset_index(drop=True, inplace=True)
            return df.to_dict('index')
            

In [203]:
Bot().nova_mensagem()

{0: {'Nome': 'Guilherme Moraes', 'Celular': '51991920910'}, 1: {'Nome': 'Sirlei', 'Celular': '(51)99761246'}}
